In [1]:
import json
import pandas as pd
import requests

In [ ]:
with open('../secrets.json', 'r') as file:
    secrets = json.load(file)

In [41]:
def get_quotes():
    params = {'token':secrets['IEX_KEY']}
    r = requests.get(f"{secrets['IEX_BASE_URL']}stable/tops", params=params)
    if r.status_code == 200:
        data = pd.DataFrame(r.json()).set_index('symbol')
        data.to_csv('../examples/IEX_quotes.csv')
    return data

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
def get_last_price():
    params = {'token':secrets['IEX_KEY']}
    r = requests.get(f"{secrets['IEX_BASE_URL']}stable/tops/last", params=params)
    if r.status_code == 200:
        data = pd.DataFrame(r.json()).set_index('symbol')
        data.to_csv('../examples/IEX_last_price.csv')
    return data

In [ ]:
# update last prices once a day at (i.e. at midnight)
data = get_last_price()

In [42]:
# update quotes regularly
data = get_quotes()

In [46]:
# , index_col='symbol'
# price is from last table and lastSalePrice is from quotes table

# merge last_price and quotes tables
df = (
    pd.read_csv('../examples/IEX_last_price.csv').merge(
        pd.read_csv('../examples/IEX_quotes.csv'),
        left_on='symbol',
        right_on='symbol'
    )
    .set_index('symbol')
    [['price', 'bidPrice', 'askPrice', 'lastSalePrice']]
)


# rename column with previous close price and drop old columns
df['prev_close'] = df['price']
df = df.drop(columns=['price'])


# clean data by removing "penny" stocks and instruments w/o bids/asks
df = df[
    (df['lastSalePrice'] > 5) # minimum $5 per share
    & (df['bidPrice'] > 0)
    & (df['askPrice'] > 0)

]


# calculate features
df['pct_change'] = df['lastSalePrice'] / df['prev_close'] - 1
df['spread_pct'] = df['askPrice'] / df['bidPrice'] - 1


# filter based on features 
df = df[
    (df['pct_change'] > .05)
    | (df['pct_change'] < -.05)
]


#df.sort_values(by='pct_change')
df.sort_values(by='spread_pct').iloc[0:50]

,bidPrice,askPrice,lastSalePrice,prev_close,pct_change,spread_pct
symbol,,,,,,
WFC,41.23,41.24,41.240,38.705,0.065495,0.000243
BAC,31.83,31.84,31.830,30.125,0.056598,0.000314
C,49.09,49.11,49.105,44.080,0.113997,0.000407
VG,20.96,20.97,20.970,19.655,0.066904,0.000477
BK,42.73,42.76,42.750,40.470,0.056338,0.000702
MCW,11.33,11.34,11.340,10.760,0.053903,0.000883
ABCL,10.49,10.50,10.495,11.100,-0.054505,0.000953
MQ,8.33,8.34,8.320,7.775,0.070096,0.001200
YMM,7.68,7.69,7.680,8.350,-0.080240,0.001302
